In [1]:

import os
os.environ["OMP_NUM_THREADS"] = "32"
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
import matplotlib.pyplot as plt
import matplotlib.cm as mpl
import seaborn as sns
import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

import dill

In [2]:
# Loading blocks...
def load_blocks(blocks):
    with open (blocks, "rb") as fh:
        bs = dill.load(fh)[0:6]
    return bs

In [10]:
g_path = '../../SBM/snakemake-layer/cache/trimmed_graph/fdr-1e-3/layered/'
tissues = ['head', 'body']
conditions = ['hs', 'ctrl']
graphs = {f'{tissue}':load_graph(g_path + f'{tissue}.xml.gz') for tissue in tissues}
b_path = '../../SBM/snakemake-layer/cache/MCMC/blocks/fdr-1e-3/layered/'
blocks = {f'{tissue}':load_blocks(b_path + f'{tissue}.dill') for tissue in tissues}

labels = [f'{tissue}-{condition}' for tissue in tissues for condition in conditions]

In [26]:
g = graphs['head']
c = "ctrl"
ds = g.ep.dataset
u = GraphView(g, efilt=lambda e: ds[e] == c)
u = Graph(u, prune=True)
u

<Graph object, undirected, with 6826 vertices and 5097528 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7f46fcb442d0>

In [28]:
cond_graphs = {}
for t in tissues:
    g = graphs[t]
    ds = g.ep.dataset
    for c in conditions:
        l = f'{t}-{c}'
        print(l)
        u = GraphView(g, efilt=lambda e: ds[e] == c)
        u = Graph(u, prune=True)  
        cond_graphs[l] = u  

In [29]:
cond_graphs

{'head-hs': <Graph object, undirected, with 6826 vertices and 4767735 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7f46fc95be10>,
 'head-ctrl': <Graph object, undirected, with 6826 vertices and 5097528 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7f46fc959190>,
 'body-hs': <Graph object, undirected, with 6575 vertices and 1303683 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7f46fc98d9d0>,
 'body-ctrl': <Graph object, undirected, with 6575 vertices and 2924794 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7f46ed716b50>}

In [30]:
state = {}
for t in tissues:
    for c in conditions:
        l = f'{t}-{c}'
        print(l)
        g = cond_graphs[l]
        bs = blocks[t]
        # Arctan transform on correlations...
        corr = g.edge_properties["spearman"]
        g.ep.weight = g.new_edge_property("double", (2*np.arctanh(corr.a)))

        # Creating nested block model...
        state[l] = NestedBlockState(g, bs=bs,
                                    state_args=dict(recs=[g.ep.weight],
                                                    rec_types=["real-normal"]))


head-hs
head-ctrl
body-hs
body-ctrl


In [31]:
for l in labels:
    s = state[l]
    g = cond_graphs[l]
    g.vp.level_0 = g.new_vertex_property("double", s.get_bs()[0])
    print(l)
    s.draw( eorder=g.ep.weight,
            # edge_pen_width = 0,
            edge_color=prop_to_size(g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 4,
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = g.vp.level_0,
            vertex_fill_color = g.vp.level_0,
            subsample_edges = 30000,
            output = "../../tmp/" + l + ".png", 
            output_size=(2000, 2000))

head-hs
head-ctrl
body-hs
body-ctrl
